In [ ]:
!hdfs dfs -mkdir /raw

In [ ]:
!hdfs dfs -mkdir /raw/new

In [4]:
!hdfs dfs -mkdir /raw/processed

In [11]:
!hadoop fs -rm -r /raw/new/*.parquet

rm: `/raw/new/*.parquet': No such file or directory


In [12]:
!hadoop fs -rm -r /raw/processed/*.parquet

Deleted /raw/processed/data_raw_20_06_2024__21_38_33.parquet


In [ ]:
!hdfs dfs -ls /raw/new/

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time
import json
from datetime import datetime

spark = SparkSession \
  .builder \
  .appName("streaming") \
  .master("local[*]") \
  .getOrCreate()

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df = spark.read.parquet("/raw/new/*.parquet")
print("Tabla estatica:", df.count())
df = df.orderBy(desc("order_id"))
df.show(truncate=False)

Tabla estatica: 217
+---------+------------------+------------------+-------------------+-----------+-----------+-----------------+------------------------------------+
|object_id|latitude          |longitude         |date               |customer_id|employee_id|quantity_products|order_id                            |
+---------+------------------+------------------+-------------------+-----------+-----------+-----------------+------------------------------------+
|339      |6.226902615623145 |-75.53320813728627|20/06/2024 21:50:33|2373       |4750       |499              |d8b9b417-b098-4344-b137-362894e4dckk|
|336      |6.302850314128526 |-75.67525212961037|20/06/2024 21:50:33|7629       |9438       |352              |d8b9b417-b098-4344-b137-362894e4dckj|
|338      |6.211841738978359 |-75.61556017602013|20/06/2024 21:50:33|3459       |2232       |264              |d8b9b417-b098-4344-b137-362894e4dcki|
|323      |6.281520982664555 |-75.54433009271456|20/06/2024 21:50:33|6569       |1473 

In [25]:
df = spark.read.parquet("/raw/new/*.parquet")
print("Tabla estatica:", df.count())
df.show(truncate=False)

Tabla estatica: 381
+---------+------------------+------------------+-------------------+-----------+-----------+-----------------+------------------------------------+
|object_id|latitude          |longitude         |date               |customer_id|employee_id|quantity_products|order_id                            |
+---------+------------------+------------------+-------------------+-----------+-----------+-----------------+------------------------------------+
|338      |6.230956920343812 |-75.62550804240905|21/06/2024 00:37:30|5110       |4942       |503              |d8b9b417-b098-4344-b137-362894e4delq|
|327      |6.280309805555333 |-75.60517544814797|21/06/2024 00:37:30|2412       |4942       |632              |d8b9b417-b098-4344-b137-362894e4delr|
|330      |6.259820848758025 |-75.56765711586326|21/06/2024 00:37:30|3521       |3830       |160              |d8b9b417-b098-4344-b137-362894e4dels|
|331      |6.239361934424389 |-75.5986138379454 |21/06/2024 00:37:30|1965       |8362 

In [26]:
import subprocess
src_path = "/raw/new/*.parquet"
dest_path = "/raw/processed/"
cmd = f"hdfs dfs -mv {src_path} {dest_path}"
result = subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)